In [1]:
import pandas as pd
import ast
import json
import base64
import numpy as np
import os
import cv2, random
np.set_printoptions(suppress=True, precision=4)
from scipy import spatial
import numpy as np
import yaml
import os.path as op
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
tqdm.pandas()

Detection

In [2]:
sg_tsv = './features_detection/training/validation.feature.tsv'
df_feature_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature_detection = df_feature_detection.rename(columns={0: "id", 1: "features"})

In [3]:
sg_tsv = './features_detection/training/validation2.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_detection=df_label_detection.rename(columns={0: "id", 1: "label"})
#df_label_detection

In [4]:
full_detection = pd.merge(df_feature_detection, df_label_detection, on="id")
#full_detection

In [5]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_detection.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

1014it [00:04, 247.69it/s]


Segmentation

In [7]:
sg_tsv = './features_test1/training/val.feature.tsv'
df_feature_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature_segmentation = df_feature_segmentation.rename(columns={0: "id", 1: "features"})

In [8]:
sg_tsv = './features_test1/training/val.label.tsv'
df_label_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_segmentation = df_label_segmentation.rename(columns={0: "id", 1: "label"})
#df_label_segmentation

In [9]:
full_segmentation = pd.merge(df_feature_segmentation, df_label_segmentation, on="id")
full_segmentation

,id,features,label
0,1018148011,{'features': 'cGlYQIDImT92q/g+AAAAABiRbEAAAAAA...,"[{'class': 'Truck', 'conf': 0.7444072961807251..."
1,1029450589,{'features': 'gq/4QCXKgEHrTThA9AsYQE5O2ED46AhA...,"[{'class': 'Sofa', 'conf': 0.9327452182769775,..."
2,1029737941,{'features': 'DGcgQcgjw0AoKupAyl+pQFG9JEEAAAAA...,"[{'class': 'Earphone', 'conf': 0.8826538324356..."
3,103205630,{'features': 'UgJZQf+q4T9LzTFAG2mFQG2ynkAAAAAA...,"[{'class': 'Tarp', 'conf': 0.7222883701324463,..."
4,10350842,{'features': 'PwGfQQAAAADDeSJAhJ0WQZboTz8oBPhA...,"[{'class': 'Life jacket', 'conf': 0.6269076466..."
...,...,...,...
1009,86120374,{'features': 'AAAAAAAAAAAAAAAA+C5ZPgAAAAASarlA...,"[{'class': 'Button', 'conf': 0.965646862983703..."
1010,86131989,{'features': '8jCEQABCxj+8tXw/NyoQQWanZT9qnYtB...,"[{'class': 'Water bottle', 'conf': 0.978633344..."
1011,91875542,{'features': 'AAAAAKQ9C0AAAAAAupY3QIHlGkGckq0/...,"[{'class': 'Hinge', 'conf': 0.9092732071876526..."
1012,981727078,{'features': 'CtMZQLFWnEHs45Q/5DnaP4V9V0BDH8FA...,"[{'class': 'Pavement', 'conf': 0.9030830860137..."


In [10]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_segmentation.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

1014it [00:01, 882.81it/s]


Union

In [11]:
full_segmentation["union"] = full_segmentation["label"] + full_detection["label"]
full_segmentation 

,id,features,label,union
0,1018148011,{'features': 'cGlYQIDImT92q/g+AAAAABiRbEAAAAAA...,"[{'class': 'Truck', 'conf': 0.7444072961807251...","[{'class': 'Truck', 'conf': 0.7444072961807251..."
1,1029450589,{'features': 'gq/4QCXKgEHrTThA9AsYQE5O2ED46AhA...,"[{'class': 'Sofa', 'conf': 0.9327452182769775,...","[{'class': 'Sofa', 'conf': 0.9327452182769775,..."
2,1029737941,{'features': 'DGcgQcgjw0AoKupAyl+pQFG9JEEAAAAA...,"[{'class': 'Earphone', 'conf': 0.8826538324356...","[{'class': 'Earphone', 'conf': 0.8826538324356..."
3,103205630,{'features': 'UgJZQf+q4T9LzTFAG2mFQG2ynkAAAAAA...,"[{'class': 'Tarp', 'conf': 0.7222883701324463,...","[{'class': 'Tarp', 'conf': 0.7222883701324463,..."
4,10350842,{'features': 'PwGfQQAAAADDeSJAhJ0WQZboTz8oBPhA...,"[{'class': 'Life jacket', 'conf': 0.6269076466...","[{'class': 'Life jacket', 'conf': 0.6269076466..."
...,...,...,...,...
1009,86120374,{'features': 'AAAAAAAAAAAAAAAA+C5ZPgAAAAASarlA...,"[{'class': 'Button', 'conf': 0.965646862983703...","[{'class': 'Button', 'conf': 0.965646862983703..."
1010,86131989,{'features': '8jCEQABCxj+8tXw/NyoQQWanZT9qnYtB...,"[{'class': 'Water bottle', 'conf': 0.978633344...","[{'class': 'Water bottle', 'conf': 0.978633344..."
1011,91875542,{'features': 'AAAAAKQ9C0AAAAAAupY3QIHlGkGckq0/...,"[{'class': 'Hinge', 'conf': 0.9092732071876526...","[{'class': 'Hinge', 'conf': 0.9092732071876526..."
1012,981727078,{'features': 'CtMZQLFWnEHs45Q/5DnaP4V9V0BDH8FA...,"[{'class': 'Pavement', 'conf': 0.9030830860137...","[{'class': 'Pavement', 'conf': 0.9030830860137..."


In [12]:
for i in tqdm(range(len(full_segmentation))):
    if(len(full_segmentation["label"].iloc[i]) + len(full_detection["label"].iloc[i]) != len(full_segmentation["union"].iloc[i])):
        print("!!!!!!!!!!!!!!!")

100%|███████████████████████████████████████████████████████████████████████████| 1014/1014 [00:00<00:00, 21169.75it/s]


In [13]:
def generate_features(x):
    idx, data, num_boxes = x["id"],x["union"],len(x["union"])
    features_arr = []
    for i in range(num_boxes):
        features = data[i]['feature']
        features_arr.append(features.astype(np.float32))
    
    features = np.vstack(tuple(features_arr))
    features = base64.b64encode(features).decode("utf-8")
    return {"features":features, "num_boxes":num_boxes}

In [14]:
def generate_labels(x):
    data = x["union"]
    res = [{"class":el['class'].capitalize(),"conf":el['conf'], "rect": el['rect']} for el in data] 
    return res

In [15]:
full_segmentation['feature_union'] = full_segmentation.progress_apply(generate_features,axis=1)
full_segmentation['feature_union'] = full_segmentation['feature_union'].progress_apply(json.dumps)

100%|█████████████████████████████████████████████████████████████████████████████| 1014/1014 [00:04<00:00, 233.87it/s]


In [16]:
full_segmentation['label_union'] = full_segmentation.progress_apply(generate_labels,axis=1)
full_segmentation['label_union'] = full_segmentation['label_union'].progress_apply(json.dumps)

100%|████████████████████████████████████████████████████████████████████████████| 1014/1014 [00:00<00:00, 2493.04it/s]


In [17]:
from features_detection.maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer

OUTPUT_DIR = './training_union_full/'
LABEL_FILE = os.path.join(OUTPUT_DIR,'validation.label.tsv')
FEATURE_FILE = os.path.join(OUTPUT_DIR,'validation.feature.tsv')
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"path to {OUTPUT_DIR} created")

In [18]:
sg_tsv = './training_union_full/validation.hw.tsv'

df_train_hw = pd.read_csv(sg_tsv,sep='\t',header = None,converters={1:json.loads})#converters={1:ast.literal_eval})
df_train_hw = df_train_hw.rename(columns={0: "id", 1: "dim"})

df_train_hw

,id,dim
0,1018148011,"[{'height': 333, 'width': 500}]"
1,1029450589,"[{'height': 334, 'width': 500}]"
2,1029737941,"[{'height': 500, 'width': 375}]"
3,103205630,"[{'height': 215, 'width': 500}]"
4,10350842,"[{'height': 375, 'width': 500}]"
...,...,...
1009,86120374,"[{'height': 500, 'width': 464}]"
1010,86131989,"[{'height': 464, 'width': 500}]"
1011,91875542,"[{'height': 375, 'width': 500}]"
1012,981727078,"[{'height': 500, 'width': 375}]"


In [19]:
tsv_writer(df_train_hw.merge(full_segmentation, on=["id"])[["id","feature_union"]].values.tolist(), FEATURE_FILE)

In [20]:
tsv_writer(df_train_hw.merge(full_segmentation, on=["id"])[["id","label_union"]].values.tolist(), LABEL_FILE)

In [ ]:
# test

In [21]:
sg_tsv = './training_union_full/validation.feature.tsv'
df_features = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_features = df_features.rename(columns={0: "id", 1: "feature"})
df_features

,id,feature
0,1018148011,{'features': 'cGlYQIDImT92q/g+AAAAABiRbEAAAAAA...
1,1029450589,{'features': 'gq/4QCXKgEHrTThA9AsYQE5O2ED46AhA...
2,1029737941,{'features': 'DGcgQcgjw0AoKupAyl+pQFG9JEEAAAAA...
3,103205630,{'features': 'UgJZQf+q4T9LzTFAG2mFQG2ynkAAAAAA...
4,10350842,{'features': 'PwGfQQAAAADDeSJAhJ0WQZboTz8oBPhA...
...,...,...
1009,86120374,{'features': 'AAAAAAAAAAAAAAAA+C5ZPgAAAAASarlA...
1010,86131989,{'features': '8jCEQABCxj+8tXw/NyoQQWanZT9qnYtB...
1011,91875542,{'features': 'AAAAAKQ9C0AAAAAAupY3QIHlGkGckq0/...
1012,981727078,{'features': 'CtMZQLFWnEHs45Q/5DnaP4V9V0BDH8FA...


In [22]:
sg_tsv = './training_union_full/validation.label.tsv'
df_labels = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_labels = df_labels.rename(columns={0: "id", 1: "label"})
df_labels

,id,label
0,1018148011,"[{'class': 'Truck', 'conf': 0.7444072961807251..."
1,1029450589,"[{'class': 'Sofa', 'conf': 0.9327452182769775,..."
2,1029737941,"[{'class': 'Earphone', 'conf': 0.8826538324356..."
3,103205630,"[{'class': 'Tarp', 'conf': 0.7222883701324463,..."
4,10350842,"[{'class': 'Life jacket', 'conf': 0.6269076466..."
...,...,...
1009,86120374,"[{'class': 'Button', 'conf': 0.965646862983703..."
1010,86131989,"[{'class': 'Water bottle', 'conf': 0.978633344..."
1011,91875542,"[{'class': 'Hinge', 'conf': 0.9092732071876526..."
1012,981727078,"[{'class': 'Pavement', 'conf': 0.9030830860137..."


In [23]:
for elem in tqdm(df_features["feature"].values): # check
    if len(np.frombuffer(base64.b64decode(elem["features"]), np.float32)) / elem["num_boxes"] != 2054:
        print("noooooooooooooooo!!!!!!!")

100%|█████████████████████████████████████████████████████████████████████████████| 1014/1014 [00:03<00:00, 326.34it/s]


In [24]:
#df_labels.merge(df_features, on=["id"])["feature"].iloc[0]
for elem, label in tqdm(zip(df_labels.merge(df_features, on=["id"])["feature"].values, df_labels.merge(df_features, on=["id"])["label"].values)): # check
    if elem["num_boxes"] != len(label):
        print("noooooooooooooooo!!!!!!!")#num_boxes

1014it [00:00, ?it/s]


In [25]:
df_labels.merge(df_features, on=["id"])["feature"].iloc[0]["num_boxes"] == len(df_labels.merge(df_features, on=["id"])["label"].iloc[0])

True